In [1]:
import importlib
resources = importlib.import_module("resources")
import resources.MonteCarlo as MC

import numpy as np
import pandas as pd
import random
random.seed(2010)

inv_f = lambda x : MC.inverse_continuous_power_law(x,1,-2)
X = MC.RandomVariable("power law, alpha = -2, for x > 1",inv_f)

testing random variable with distribution power law, alpha = -2, for x > 1,
 1.1616517020703265

